<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# $\texttt{GiRaFFE}$: General Relativistic Force-Free Electrodynamics

## Authors: Patrick Nelson, Zachariah Etienne, George Vopal & Maria Babiuc-Hamilton
### Formatting improvements courtesy Brandon Clark

## GiRaFFE_HO_v2 is an experiment that omits the analytic derivatives of $\partial_j T^j_{{\rm EM} i}$


**Notebook Status:** <font color='red'><b> In progress </b></font>

**Validation Notes:** This module is under active development -- do ***not*** use the resulting C code output for doing science.


### NRPy+ Source Code for this module: [GiRaFFE_HO/GiRaFFE_Higher_Order_v2.py](../edit/GiRaFFE_HO/GiRaFFE_Higher_Order_v2.py)

## Introduction:
The original $\texttt{GiRaFFE}$ code, as presented in [the original paper](https://arxiv.org/pdf/1704.00599.pdf), exists as a significant modification to $\texttt{IllinoisGRMHD}$. As such, it used a third-order reconstruction algorithm with a slope limiter (Colella et al's piecewise parabolic method, or PPM) to handle spatial derivatives in the general relativistic force-free electrodynamics (GRFFE) equations. However the GRFFE equations do not generally permit shocks, so a more optimal approach would involve finite differencing all derivatives in the GRFFE equations. As it happens, NRPy+ was designed to generate C codes involving complex tensorial expressions and finite difference spatial derivatives, with finite-differencing order a freely-specifiable parameter.

The purpose of this notebook is to rewrite the equations of GRFFE as used in the original $\texttt{GiRaFFE}$ code so that all derivatives that appear are represented numerically as finite difference derivatives. 

The GRFFE evolution equations (from eq. 13 of the [original paper](https://arxiv.org/pdf/1704.00599.pdf)) we wish to encode in the NRPy+ version of $\texttt{GiRaFFE}$ are as follows:

* $\partial_t \tilde{S}_i = - \partial_j \left( \alpha \sqrt{\gamma} T^j_{{\rm EM} i} \right) + \frac{1}{2} \alpha \sqrt{\gamma} T^{\mu \nu}_{\rm EM} \partial_i g_{\mu \nu}$: [Link to Step 3.0](#step7)
* $\partial_t A_i = \epsilon_{ijk} v^j B^k - \partial_i (\alpha \Phi - \beta^j A_j)$: [Link to Step 4.0](#step8)
* $\partial_t [\sqrt{\gamma} \Phi] = -\partial_j (\alpha\sqrt{\gamma}A^j - \beta^j [\sqrt{\gamma} \Phi]) - \xi \alpha [\sqrt{\gamma} \Phi]$: [Link to Step 4.0](#step8)

Here, the densitized spatial Poynting flux one-form $\tilde{S}_i = \sqrt{\gamma} S_i$ (and $S_i$ comes from $S_{\mu} -n_{\nu} T^{\nu}_{{\rm EM} \mu}$), and $(\Phi, A_i)$ is the vector potential. We will solve these PDEs using the method of lines, where the right-hand sides of these equations (involving no explicit time derivatives) will be constructed using NRPy+.

### A Note on Notation:

As is standard in NRPy+, 

* Greek indices refer to four-dimensional quantities where the zeroth component indicates temporal (time) component.
* Latin indices refer to three-dimensional quantities. This is somewhat counterintuitive since Python always indexes its lists starting from 0. As a result, the zeroth component of three-dimensional quantities will necessarily indicate the first *spatial* direction.

For instance, in calculating the first term of [$T_{\rm EM}^{\mu\nu}$](#em_tensor) (specifically, ${\rm Term\ 1} = b^2 u^\mu u^\nu$), we use Greek indices:

```
T4EMUU = ixp.zerorank2(DIM=4)
for mu in range(4):
    for nu in range(4):
        # Term 1: b^2 u^{\mu} u^{\nu}
        T4EMUU[mu][nu] = smallb2*u4U[mu]*u4U[nu]\
```

When we calculate [$\beta_i = \gamma_{ij} \beta^j$](#4metric), we use Latin indices:
```
betaD = ixp.zerorank1()
for i in range(DIM):
    for j in range(DIM):
        betaD[i] += gammaDD[i][j] * betaU[j]
```

As a corollary, any expressions involving mixed Greek and Latin indices will need to offset one set of indices by one: A Latin index in a four-vector will be incremented and a Greek index in a three-vector will be decremented (however, the latter case does not occur in this tutorial notebook). This can be seen when we handle the second term of [$\partial_t \tilde{S}_i$](#construct_si) (or, more specifically, the second term thereof: $\frac{1}{2} \alpha \sqrt{\gamma} T^{\mu \nu}_{\rm EM} \partial_i g_{\mu \nu}$):
```
# The second term: \alpha \sqrt{\\gamma} T^{\mu \nu}_{\rm EM} \partial_i g_{\mu \nu} / 2
for i in range(DIM):
    for mu in range(4):
        for nu in range(4):
            Stilde_rhsD[i] += alpsqrtgam * T4EMUU[mu][nu] * g4DDdD[mu][nu][i+1] / 2
```

<a id='toc'></a>

# Table of Contents:
$$\label{toc}$$

This notebook is organized as follows

1. [Step 1](#initializenrpy): Set up the needed NRPy+ infrastructure and declare core gridfunctions used by $\texttt{GiRaFFE}$
1. [Step 2](#4metric): Build the four metric $g_{\mu\nu}$, its inverse $g^{\mu\nu}$ and spatial derivatives $g_{\mu\nu,i}$ from ADM 3+1 quantities $\gamma_{ij}$, $\beta^i$, and $\alpha$
1. [Step 3](#t_derivatives): $T^{\mu\nu}_{\rm EM}$ and its derivatives
    1. [Step 3.a](#uibi): $u^i$ and $b^i$ and related quantities
    1. [Step 3.b](#em_tensor): Construct all components of the electromagnetic stress-energy tensor $T^{\mu \nu}_{\rm EM}$
1. [Step 4](#construct_si): Construct the evolution equation for $\tilde{S}_i$
1. [Step 5](#construct_ai): Construct the evolution equations for $A_i$ and $[\sqrt{\gamma}\Phi]$
    1. [Step 5.a](#aux): Construct some useful auxiliary gridfunctions for the other evolution equations
    1. [Step 5.b](#complete_construct_ai): Complete the construction of the evolution equations for $A_i$ and $[\sqrt{\gamma}\Phi]$
1. [Step 6](#code_validation): Code Validation against `GiRaFFE_HO.GiRaFFE_Higher_Order_v2` NRPy+ Module
1. [Step 7](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

<a id='initializenrpy'></a>

# Step 1: Set up the needed NRPy+ infrastructure and declare core gridfunctions used by $\texttt{GiRaFFE}$ \[Back to [top](#toc)\]
$$\label{initializenrpy}$$


1. Set some basic NRPy+ parameters. E.g., set the spatial dimension parameter to 3 and the finite differencing order to 4.
1. Next, declare some gridfunctions that are provided as input to the equations:
    1. $\alpha$, $\beta^i$, and $\gamma_{ij}$: These ADM 3+1 metric quantities are declared in the ADMBase Einstein Toolkit thorn, and are assumed to be made available to $\texttt{GiRaFFE}$ at this stage.
    1. The Valencia 3-velocity $v^i_{(n)}$ and vector potential $A_i$: Declared by $\texttt{GiRaFFE}$, and will have their initial values set in the separate thorn **GiRaFFEfood_HO**.
    1. The magnetic field as measured by a normal observer $B^i$: The quantities evolved forward in time in $\texttt{GiRaFFE}$ do not include the Valencia 3-velocity, so this quantity is not automatically updated. Instead, we compute it based on the evolved quantity $\tilde{S}_i$ and $B^i = \epsilon^{ijk} \partial_j A_k$ (where $A_k$ is another evolved quantity and $\epsilon^{ijk}$ is the Levi-Civita tensor). $B^i$ is evaluated using finite differences of $A_k$ in a separate function, though it can only be evaluated consistently on the interior of the grid. In the ghost zones, we will have to use lower-order derivatives.


In [1]:
# Step 0: Add NRPy's directory to the path
# https://stackoverflow.com/questions/16780014/import-file-from-parent-directory
import os,sys
nrpy_dir_path = os.path.join("..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)

import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
import finite_difference as fin
from outputC import *

#Step 1.0: Set the spatial dimension parameter to 3.
par.set_parval_from_str("grid::DIM", 3)
DIM = par.parval_from_str("grid::DIM")

# Step 1.1: Set the finite differencing order to 4.
par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 4)

thismodule = "GiRaFFE_NRPy"

# M_PI will allow the C code to substitute the correct value
M_PI = par.Cparameters("#define",thismodule,"M_PI","")
# ADMBase defines the 4-metric in terms of the 3+1 spacetime metric quantities gamma_{ij}, beta^i, and alpha
gammaDD = ixp.register_gridfunctions_for_single_rank2("AUX","gammaDD", "sym01",DIM=3)
betaU   = ixp.register_gridfunctions_for_single_rank1("AUX","betaU",DIM=3)
alpha   = gri.register_gridfunctions("AUX","alpha")
# GiRaFFE uses the Valencia 3-velocity and A_i, which are defined in the initial data module(GiRaFFEfood)
ValenciavU = ixp.register_gridfunctions_for_single_rank1("AUX","ValenciavU",DIM=3)
AD = ixp.register_gridfunctions_for_single_rank1("EVOL","AD",DIM=3)
# B^i must be computed at each timestep within GiRaFFE so that the Valencia 3-velocity can be evaluated
BU = ixp.register_gridfunctions_for_single_rank1("AUX","BU",DIM=3)

<a id='4metric'></a>

# Step 2: Build the four metric $g_{\mu\nu}$, its inverse $g^{\mu\nu}$ and spatial derivatives $g_{\mu\nu,i}$ from ADM 3+1 quantities $\gamma_{ij}$, $\beta^i$, and $\alpha$ \[Back to [top](#toc)\]
$$\label{4metric}$$

Notice that the time evolution equation for $\tilde{S}_i$
$$
\partial_t \tilde{S}_i = - \partial_j \left( \alpha \sqrt{\gamma} T^j_{{\rm EM} i} \right) + \frac{1}{2} \alpha \sqrt{\gamma} T^{\mu \nu}_{\rm EM} \partial_i g_{\mu \nu}
$$
contains $\partial_i g_{\mu \nu} = g_{\mu\nu,i}$. We will now focus on evaluating this term.

The four-metric $g_{\mu\nu}$ is related to the three-metric $\gamma_{ij}$, index-lowered shift $\beta_i$, and lapse $\alpha$ by  
$$
g_{\mu\nu} = \begin{pmatrix} 
-\alpha^2 + \beta^k \beta_k & \beta_j \\
\beta_i & \gamma_{ij}
\end{pmatrix}.
$$
This tensor and its inverse have already been built by the u0_smallb_Poynting__Cartesian.py module ([documented here](Tutorial-u0_smallb_Poynting-Cartesian.ipynb)), so we can simply load the module and import the variables.

In [2]:
# Step 1.2: import u0_smallb_Poynting__Cartesian.py to set
#           the four metric and its inverse. This module also sets b^2 and u^0.
import u0_smallb_Poynting__Cartesian.u0_smallb_Poynting__Cartesian as u0b
u0b.compute_u0_smallb_Poynting__Cartesian(gammaDD,betaU,alpha,ValenciavU,BU)

betaD = ixp.zerorank1()
for i in range(DIM):
    for j in range(DIM):
        betaD[i] += gammaDD[i][j] * betaU[j]
# Error check: fixed = to +=

# We will now pull in the four metric and its inverse.
import BSSN.ADMBSSN_tofrom_4metric as AB4m
gammaDD,betaU,alpha = AB4m.setup_ADM_quantities("ADM")
AB4m.g4DD_ito_BSSN_or_ADM("ADM",gammaDD,betaU,alpha)
g4DD = AB4m.g4DD
AB4m.g4UU_ito_BSSN_or_ADM("ADM",gammaDD,betaU,alpha)
g4UU = AB4m.g4UU

Next we compute spatial derivatives of the metric, $\partial_i g_{\mu\nu} = g_{\mu\nu,i}$, written in terms of the three-metric, shift, and lapse. Simply taking the derivative of the expression for $g_{\mu\nu}$ above, we find
$$
g_{\mu\nu,l} = \begin{pmatrix} 
-2\alpha \alpha_{,l} + \beta^k_{\ ,l} \beta_k + \beta^k \beta_{k,l} & \beta_{j,l} \\
\beta_{i,l} & \gamma_{ij,l}
\end{pmatrix}.
$$

Notice the derivatives of the shift vector with its indexed lowered, $\beta_{i,j} = \partial_j \beta_i$. This can be easily computed in terms of the given ADMBase quantities $\beta^i$ and $\gamma_{ij}$ via:
\begin{align}
\beta_{i,j} &= \partial_j \beta_i \\
            &= \partial_j (\gamma_{ik} \beta^k) \\
            &= \gamma_{ik} \partial_j\beta^k + \beta^k \partial_j \gamma_{ik} \\
\beta_{i,j} &= \gamma_{ik} \beta^k_{\ ,j} + \beta^k \gamma_{ik,j}.
\end{align}

Since this expression mixes Greek and Latin indices, we will declare this as a 4 dimensional quantity, but only set the three spatial components of its last index (that is, leaving $l=0$ unset).

So, we will first set 
$$ g_{00,l} = \underbrace{-2\alpha \alpha_{,l}}_{\rm Term\ 1} + \underbrace{\beta^k_{\ ,l} \beta_k}_{\rm Term\ 2} + \underbrace{\beta^k \beta_{k,l}}_{\rm Term\ 3} $$

In [3]:
# Step 1.2, cont'd: Build spatial derivatives of the four metric
# Step 1.2.a: Declare derivatives of grid functions. These will be handled by FD_outputC
alpha_dD   = ixp.declarerank1("alpha_dD")
betaU_dD   = ixp.declarerank2("betaU_dD","nosym")
gammaDD_dD = ixp.declarerank3("gammaDD_dD","sym01")

# Step 1.2.b: These derivatives will be constructed analytically.
betaDdD    = ixp.zerorank2()
g4DDdD     = ixp.zerorank3(DIM=4)

for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            # \gamma_{ik} \beta^k_{,j} + \beta^k \gamma_{ik,j}
            betaDdD[i][j] += gammaDD[i][k] * betaU_dD[k][j] + betaU[k] * gammaDD_dD[i][k][j]
#Error check: changed = to += above

# Step 1.2.c: Set the 00 components
# Step 1.2.c.i: Term 1: -2\alpha \alpha_{,l}
for l in range(DIM):
    g4DDdD[0][0][l+1] = -2*alpha*alpha_dD[l]

# Step 1.2.c.ii: Term 2: \beta^k_{\ ,l} \beta_k
for l in range(DIM):
    for k in range(DIM):
        g4DDdD[0][0][l+1] += betaU_dD[k][l] * betaD[k]

# Step 1.2.c.iii: Term 3: \beta^k \beta_{k,l}
for l in range(DIM):
    for k in range(DIM):
        g4DDdD[0][0][l+1] += betaU[k] * betaDdD[k][l]

Now we will contruct the other components of $g_{\mu\nu,l}$. We will first construct 
$$ g_{i0,l} = g_{0i,l} = \beta_{i,l}, $$
then
$$ g_{ij,l} = \gamma_{ij,l} $$

In [4]:
# Step 1.2.d: Set the i0 and 0j components
for l in range(DIM):
    for i in range(DIM):
        # \beta_{i,l}
        g4DDdD[i+1][0][l+1] = g4DDdD[0][i+1][l+1] = betaDdD[i][l]

#Step 1.2.e: Set the ij components
for l in range(DIM):
    for i in range(DIM):
        for j in range(DIM):
            # \gamma_{ij,l}
            g4DDdD[i+1][j+1][l+1] = gammaDD_dD[i][j][l]

<a id='t_derivatives'></a>

# Step 3: $T^{\mu\nu}_{\rm EM}$ and its derivatives \[Back to [top](#toc)\]
$$\label{t_derivatives}$$

Now that the metric and its derivatives are out of the way, let's return to the evolution equation for $\tilde{S}_i$,
$$
\partial_t \tilde{S}_i = - \partial_j \left( \alpha \sqrt{\gamma} T^j_{{\rm EM} i} \right) + \frac{1}{2} \alpha \sqrt{\gamma} T^{\mu \nu}_{\rm EM} \partial_i g_{\mu \nu}.
$$ 
We turn our focus now to $T^j_{{\rm EM} i}$ and its derivatives. To this end, we start by computing $T^{\mu \nu}_{\rm EM}$ (from eq. 27 of [Paschalidis & Shapiro's paper on their GRFFE code](https://arxiv.org/pdf/1310.3274.pdf)):

$$\boxed{T^{\mu \nu}_{\rm EM} = b^2 u^{\mu} u^{\nu} + \frac{b^2}{2} g^{\mu \nu} - b^{\mu} b^{\nu}.}$$

Notice that $T^{\mu\nu}_{\rm EM}$ is written in terms of

* $b^\mu$, the 4-component magnetic field vector, related to the comoving magnetic field vector $B^i_{(u)}$
* $u^\mu$, the 4-velocity
* $g^{\mu \nu}$, the inverse 4-metric

However, $\texttt{GiRaFFE}$ has access to only the following quantities, requiring in the following sections that we write the above quantities in terms of the following ones:

* $\gamma_{ij}$, the 3-metric
* $\alpha$, the lapse
* $\beta^i$, the shift
* $A_i$, the vector potential
* $B^i$, the magnetic field (we assume only in the grid interior, not the ghost zones)
* $\left[\sqrt{\gamma}\Phi\right]$, the zero-component of the vector potential $A_\mu$, times the square root of the determinant of the 3-metric
* $v_{(n)}^i$, the Valencia 3-velocity
* $u^0$, the zero-component of the 4-velocity

<a id='uibi'></a>

## Step 3.a: $u^i$ and $b^i$ and related quantities \[Back to [top](#toc)\]
$$\label{uibi}$$

We begin by importing what we can from [u0_smallb_Poynting__Cartesian.py](../edit/u0_smallb_Poynting__Cartesian/u0_smallb_Poynting__Cartesian.py). We will need the four-velocity $u^\mu$, which is related to the Valencia 3-velocity $v^i_{(n)}$ used directly by $\texttt{GiRaFFE}$ (see also [Duez, et al, eqs. 53 and 56](https://arxiv.org/pdf/astro-ph/0503420.pdf))
\begin{align}
u^i &= u^0 (\alpha v^i_{(n)} - \beta^i), \\
u_j &= \alpha u^0 \gamma_{ij} v^i_{(n)},
\end{align}
where $v^i_{(n)}$ is the Valencia three-velocity. These have already been constructed in terms of the Valencia 3-velocity and other 3+1 ADM quantities by the [u0_smallb_Poynting__Cartesian.py](../edit/u0_smallb_Poynting__Cartesian/u0_smallb_Poynting__Cartesian.py) module, so we can simply import these variables:


In [5]:
# Step 2.0: u^i, b^i, and related quantities
# Step 2.0.a: import the four-velocity, as written in terms of the Valencia 3-velocity
uD = ixp.zerorank1()
uU = ixp.zerorank1()
u4upperZero = gri.register_gridfunctions("AUX","u4upperZero")

for i in range(DIM):
    uD[i] = u0b.uD[i].subs(u0b.u0,u4upperZero)
    uU[i] = u0b.uU[i].subs(u0b.u0,u4upperZero)


We also need the magnetic field 4-vector $b^{\mu}$, which is related to the magnetic field by [eqs. 23, 24, and 31 in Duez, et al](https://arxiv.org/pdf/astro-ph/0503420.pdf):
\begin{align}
b^0 &= \frac{1}{\sqrt{4\pi}} B^0_{\rm (u)} = \frac{u_j B^j}{\sqrt{4\pi}\alpha}, \\
b^i &= \frac{1}{\sqrt{4\pi}} B^i_{\rm (u)} = \frac{B^i + (u_j B^j) u^i}{\sqrt{4\pi}\alpha u^0}, \\
\end{align}
where $B^i$ is the variable tracked by the HydroBase thorn in the Einstein Toolkit. Again, these have already been built by the [u0_smallb_Poynting__Cartesian.py](../edit/u0_smallb_Poynting__Cartesian/u0_smallb_Poynting__Cartesian.py), so we can simply import the variables.

In [6]:
# Step 2.0.b: import the small b terms
smallb4U = ixp.zerorank1(DIM=4)
smallb4D = ixp.zerorank1(DIM=4)
for mu in range(4):
    smallb4U[mu] = u0b.smallb4U[mu].subs(u0b.u0,u4upperZero)
    smallb4D[mu] = u0b.smallb4D[mu].subs(u0b.u0,u4upperZero)

smallb2 = u0b.smallb2etk.subs(u0b.u0,u4upperZero)


<a id='em_tensor'></a>

## Step 3.b: Construct all components of the electromagnetic stress-energy tensor $T^{\mu \nu}_{\rm EM}$ \[Back to [top](#toc)\]
$$\label{em_tensor}$$

We now have all the pieces to calculate the stress-energy tensor,
$$T^{\mu \nu}_{\rm EM} = \underbrace{b^2 u^{\mu} u^{\nu}}_{\rm Term\ 1} + 
\underbrace{\frac{b^2}{2} g^{\mu \nu}}_{\rm Term\ 2}
- \underbrace{b^{\mu} b^{\nu}}_{\rm Term\ 3}.$$
Because $u^0$ is a separate variable, we will create a temporary variable $u^\mu=\left( u^0, u^i \right)$,

In [7]:
# Step 2.1: Construct the electromagnetic stress-energy tensor
# Step 2.1.a: Set up the four-velocity vector
u4U = ixp.zerorank1(DIM=4)
u4U[0] = u4upperZero
for i in range(DIM):
    u4U[i+1] = uU[i]

# Step 2.1.b: Build T4EMUU itself
T4EMUU = ixp.zerorank2(DIM=4)
for mu in range(4):
    for nu in range(4):
        # Term 1: b^2 u^{\mu} u^{\nu}
        T4EMUU[mu][nu] = smallb2*u4U[mu]*u4U[nu]

for mu in range(4):
    for nu in range(4):
        # Term 2: b^2 / 2 g^{\mu \nu}
        T4EMUU[mu][nu] += smallb2*g4UU[mu][nu]/2

for mu in range(4):
    for nu in range(4):
        # Term 3: -b^{\mu} b^{\nu}
        T4EMUU[mu][nu] += -smallb4U[mu]*smallb4U[nu]

<a id='construct_si'></a>

# Step 4: Construct the evolution equation for $\tilde{S}_i$ \[Back to [top](#top)\]
$$\label{construct_si}$$

Finally, we will return our attention to the time evolution equation (from eq. 13 of the [original paper](https://arxiv.org/pdf/1704.00599.pdf)),
\begin{align}
\partial_t \tilde{S}_i &= - \partial_j \underbrace{\left( \alpha \sqrt{\gamma} T^j_{{\rm EM} i} \right)}_{\rm SevolParenUD} + \frac{1}{2} \alpha \sqrt{\gamma} T^{\mu \nu}_{\rm EM} \partial_i g_{\mu \nu} \\
                       &= - \partial_j{\rm SevolParenUD[j][i]} + \frac{1}{2} \alpha \sqrt{\gamma} T^{\mu \nu}_{\rm EM} \partial_i g_{\mu \nu} .
\end{align}
We will first construct `SevolParenUD`, then use its derivatives to construct the evolution equation. Note that 
\begin{align}
{\rm SevolParenUD[j][i]} &= \alpha \sqrt{\gamma} T^j_{{\rm EM} i} \\
                             &= \alpha \sqrt{\gamma} g_{\mu i} T^{\mu j}_{\rm EM}.
\end{align}

In [8]:
# Step 3.0: Construct the evolution equation for \tilde{S}_i
# Here, we set up the necessary machinery to take FD derivatives of alpha * sqrt(gamma)
gammaUU = ixp.register_gridfunctions_for_single_rank2("AUX","gammaUU","sym01")
gammadet = gri.register_gridfunctions("AUX","gammadet")
gammaUU, gammadet = ixp.symm_matrix_inverter3x3(gammaDD)

alpsqrtgam = alpha*sp.sqrt(gammadet)

SevolParenUD = ixp.register_gridfunctions_for_single_rank2("AUX","SevolParenUD","nosym")
SevolParenUD = ixp.zerorank2()

for i in range(DIM):
    for j in range (DIM):
        for mu in range(4):
            SevolParenUD[j][i] += alpsqrtgam * g4DD[mu][i+1] * T4EMUU[mu][j+1]

SevolParenUD_dD = ixp.declarerank3("SevolParenUD_dD","nosym")

Stilde_rhsD = ixp.zerorank1()
# The first term: \alpha \sqrt{\gamma} \partial_j T^j_{{\rm EM} i}
for i in range(DIM):
    for j in range(DIM):
        Stilde_rhsD[i] += -SevolParenUD_dD[j][i][j]

# The second term: \alpha \sqrt{\gamma} T^{\mu \nu}_{\rm EM} \partial_i g_{\mu \nu} / 2
for i in range(DIM):
    for mu in range(4):
        for nu in range(4):
            Stilde_rhsD[i] += alpsqrtgam * T4EMUU[mu][nu] * g4DDdD[mu][nu][i+1] / 2


<a id='construct_ai'></a>

# Step 5: Construct the evolution equations for $A_i$ and $[\sqrt{\gamma}\Phi]$ \[Back to [top](#toc)\]
$$\label{construct_ai}$$

We will also need to evolve the vector potential $A_i$. This evolution is given as eq. 17 in the [$\texttt{GiRaFFE}$](https://arxiv.org/pdf/1704.00599.pdf) paper:
$$\boxed{\partial_t A_i = \epsilon_{ijk} v^j B^k - \partial_i (\underbrace{\alpha \Phi - \beta^j A_j}_{\rm AevolParen}),}$$
where $\epsilon_{ijk} = [ijk] \sqrt{\gamma}$ is the antisymmetric Levi-Civita tensor, the drift velocity $v^i = u^i/u^0$, $\gamma$ is the determinant of the three metric, $B^k$ is the magnetic field, $\alpha$ is the lapse, and $\beta$ is the shift.

We also need the scalar electric potential $\Phi$, which is evolved by eq. 19:
$$\boxed{\partial_t [\sqrt{\gamma} \Phi] = -\partial_j (\underbrace{\alpha\sqrt{\gamma} \gamma^{ij} A_i - \beta^j [\sqrt{\gamma} \Phi]}_{\rm PevolParenU[j]}) - \xi \alpha [\sqrt{\gamma} \Phi],}$$
with $\xi$ chosen as a damping factor. 

<a id='aux'></a>

## Step 5.a: Construct some useful auxiliary gridfunctions for the other evolution equations \[Back to [top](#toc)\]
$$\label{aux}$$

After declaring a  some needed quantities, we will also define the parenthetical terms (underbrace above) that we need to take derivatives of. That way, we can take finite-difference derivatives easily. Note that the above equations incorporate the fact that $\gamma^{ij}$ is the appropriate raising operator for $A_i$: $A^j = \gamma^{ij} A_i$. This is correct since $n_\mu A^\mu = 0$, where $n_\mu$ is a normal to the hypersurface, so $A^0=0$ (according to Sec. II, subsection C of [the "Improved EM gauge condition" paper of Etienne *et al*](https://arxiv.org/pdf/1110.4633.pdf)).

In [9]:
# Step 4.0: Construct the evolution equations for A_i and sqrt(gamma)Phi
# Step 4.0.a: Construct some useful auxiliary gridfunctions for the other evolution equations
xi = par.Cparameters("REAL",thismodule,"xi",0.1) # The damping factor

# Define sqrt(gamma)Phi as psi6Phi
psi6Phi = gri.register_gridfunctions("EVOL","psi6Phi")
Phi = psi6Phi / sp.sqrt(gammadet)

# We'll define a few extra gridfunctions to avoid complicated derivatives
AevolParen  = gri.register_gridfunctions("AUX","AevolParen")
PevolParenU = ixp.register_gridfunctions_for_single_rank1("AUX","PevolParenU")

# {\rm AevolParen} = \alpha \Phi - \beta^j A_j
AevolParen = alpha*Phi
for j in range(DIM):
    AevolParen += -betaU[j] * AD[j]

# {\rm PevolParenU[j]} = \alpha\sqrt{\gamma} \gamma^{ij} A_i - \beta^j [\sqrt{\gamma} \Phi]
for j in range(DIM):
    PevolParenU[j] = -betaU[j] * psi6Phi
    for i in range(DIM):
        PevolParenU[j] += alpsqrtgam * gammaUU[i][j] * AD[i]

AevolParen_dD  = ixp.declarerank1("AevolParen_dD")
PevolParenU_dD = ixp.declarerank2("PevolParenU_dD","nosym")


<a id='complete_construct_ai'></a>

## Step 5.b: Complete the construction of the evolution equations for $A_i$ and $[\sqrt{\gamma}\Phi]$ \[Back to [top](#toc)\]
$$\label{complete_construct_ai}$$

Now to set the evolution equations ([eqs. 17 and 19](https://arxiv.org/pdf/1704.00599.pdf)), recalling that the drift velocity $v^i = u^i/u^0$:
\begin{align}
\partial_t A_i &= \epsilon_{ijk} v^j B^k - \partial_i (\alpha \Phi - \beta^j A_j) \\
               &= \epsilon_{ijk} \frac{u^j}{u^0} B^k - {\rm AevolParen\_dD[i]} \\
\partial_t [\sqrt{\gamma} \Phi] &= -\partial_j \left(\left(\alpha\sqrt{\gamma}\right)A^j - \beta^j [\sqrt{\gamma} \Phi]\right) - \xi \alpha [\sqrt{\gamma} \Phi] \\
                                &= -{\rm PevolParenU\_dD[j][j]} - \xi \alpha [\sqrt{\gamma} \Phi]. \\
\end{align}

In [10]:
# Step 4.0.b: Construct the actual evolution equations for A_i and sqrt(gamma)Phi
A_rhsD = ixp.zerorank1()
psi6Phi_rhs = sp.sympify(0)

# We already have a handy function to define the Levi-Civita symbol in WeylScalars
import WeylScal4NRPy.WeylScalars_Cartesian as weyl
# Initialize the Levi-Civita tensor by setting it equal to the Levi-Civita symbol
LeviCivitaSymbolDDD = weyl.define_LeviCivitaSymbol_rank3()
LeviCivitaTensorDDD = ixp.zerorank3()
#LeviCivitaTensorUUU = ixp.zerorank3()

for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            LeviCivitaTensorDDD[i][j][k] = LeviCivitaSymbolDDD[i][j][k] * sp.sqrt(gammadet)
            #LeviCivitaTensorUUU[i][j][k] = LeviCivitaSymbolDDD[i][j][k] / sp.sqrt(gammadet)

for i in range(DIM):
    A_rhsD[i] = -AevolParen_dD[i]
    for j in range(DIM):
        for k in range(DIM):
            A_rhsD[i] += LeviCivitaTensorDDD[i][j][k]*(uU[j]/u4upperZero)*BU[k]

psi6Phi_rhs = -xi*alpha*psi6Phi
for j in range(DIM):
    psi6Phi_rhs += -PevolParenU_dD[j][j]

<a id='code_validation'></a>

# Step 6:  Code Validation against `GiRaFFE_HO.GiRaFFE_Higher_Order_v2` NRPy+ Module \[Back to [top](#toc)\]
$$\label{code_validation}$$


Here, as a code validation check, we verify agreement in the SymPy expressions for the $\texttt{GiRaFFE}$ evolution equations and auxiliary quantities we intend to use between
1. this tutorial and 
2. the NRPy+ [GiRaFFE_HO.GiRaFFE_Higher_Order_v2](../edit/GiRaFFE_HO/GiRaFFE_Higher_Order_v2.py) module.


In [11]:
# Reset the list of gridfunctions, as registering a gridfunction
#   twice will spawn an error.
gri.glb_gridfcs_list = []

import GiRaFFE_HO.GiRaFFE_Higher_Order_v2 as gho
gho.GiRaFFE_Higher_Order_v2()

print("Consistency check between GiRaFFE_Higher_Order tutorial and NRPy+ module: ALL SHOULD BE ZERO.")

print("AevolParen - gho.AevolParen = " + str(AevolParen - gho.AevolParen))
print("psi6Phi_rhs - gho.psi6Phi_rhs = " + str(psi6Phi_rhs - gho.psi6Phi_rhs))
for i in range(DIM):
    print("PevolParenU["+str(i)+"] - gho.PevolParenU["+str(i)+"] = " + str(PevolParenU[i] - gho.PevolParenU[i]))
    print("Stilde_rhsD["+str(i)+"] - gho.Stilde_rhsD["+str(i)+"] = " + str(Stilde_rhsD[i] - gho.Stilde_rhsD[i]))
    print("A_rhsD["+str(i)+"] - gho.A_rhsD["+str(i)+"] = " + str(A_rhsD[i] - gho.A_rhsD[i]))
    for j in range(DIM):
        print("SevolParenUD["+str(i)+"]["+str(j)+"] - gho.SevolParenUD["+str(i)+"]["+str(i)+"] = "\
              + str(SevolParenUD[i][j] - gho.SevolParenUD[i][j]))

Consistency check between GiRaFFE_Higher_Order tutorial and NRPy+ module: ALL SHOULD BE ZERO.
AevolParen - gho.AevolParen = 0
psi6Phi_rhs - gho.psi6Phi_rhs = 0
PevolParenU[0] - gho.PevolParenU[0] = 0
Stilde_rhsD[0] - gho.Stilde_rhsD[0] = 0
A_rhsD[0] - gho.A_rhsD[0] = 0
SevolParenUD[0][0] - gho.SevolParenUD[0][0] = 0
SevolParenUD[0][1] - gho.SevolParenUD[0][0] = 0
SevolParenUD[0][2] - gho.SevolParenUD[0][0] = 0
PevolParenU[1] - gho.PevolParenU[1] = 0
Stilde_rhsD[1] - gho.Stilde_rhsD[1] = 0
A_rhsD[1] - gho.A_rhsD[1] = 0
SevolParenUD[1][0] - gho.SevolParenUD[1][1] = 0
SevolParenUD[1][1] - gho.SevolParenUD[1][1] = 0
SevolParenUD[1][2] - gho.SevolParenUD[1][1] = 0
PevolParenU[2] - gho.PevolParenU[2] = 0
Stilde_rhsD[2] - gho.Stilde_rhsD[2] = 0
A_rhsD[2] - gho.A_rhsD[2] = 0
SevolParenUD[2][0] - gho.SevolParenUD[2][2] = 0
SevolParenUD[2][1] - gho.SevolParenUD[2][2] = 0
SevolParenUD[2][2] - gho.SevolParenUD[2][2] = 0


<a id='latex_pdf_output'></a>

# Step 7: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-GiRaFFE_Higher_Order_v2.pdf](Tutorial-GiRaFFE_Higher_Order_v2.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [12]:
!jupyter nbconvert --to latex --template latex_nrpy_style.tplx --log-level='WARN' Tutorial-GiRaFFE_Higher_Order_v2.ipynb
!pdflatex -interaction=batchmode Tutorial-GiRaFFE_Higher_Order_v2.tex
!pdflatex -interaction=batchmode Tutorial-GiRaFFE_Higher_Order_v2.tex
!pdflatex -interaction=batchmode Tutorial-GiRaFFE_Higher_Order_v2.tex
!rm -f Tut*.out Tut*.aux Tut*.log

[NbConvertApp] Converting notebook Tutorial-GiRaFFE_Higher_Order_v2.ipynb to latex
[NbConvertApp] Writing 77392 bytes to Tutorial-GiRaFFE_Higher_Order_v2.tex
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
